In [0]:
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "<client-id>",
    "fs.azure.account.oauth2.client.secret": "<secret-value>",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant-id>/oauth2/token"
}

dbutils.fs.mount(
    source="abfss://tokyo-olympic-data@mayvalutofdp203.dfs.core.windows.net/",
    mount_point="/mnt/tokyo-olympic",
    extra_configs=configs
    )

Out[8]: True

In [0]:
%fs
ls "/mnt/tokyo-olympic"

path,name,size,modificationTime
dbfs:/mnt/tokyo-olympic/raw-data/,raw-data/,0,1732394730000
dbfs:/mnt/tokyo-olympic/transformed-data/,transformed-data/,0,1732394743000


In [0]:
# Define a fucntion to load CSV Data 

def load_csv_data(spark, file_path, header=True, inferSchema=True, encoding="UTF-8"):
    return spark.read.format("csv").option("header", header).option("inferSchema", inferSchema).option("encoding", encoding).load(file_path)


In [0]:
# Load Datasets using the above fundtion

athletes = load_csv_data(spark, "/mnt/tokyo-olympic/raw-data/athletes.csv")
coaches = load_csv_data(spark, "/mnt/tokyo-olympic/raw-data/coaches.csv")
entriesgender = load_csv_data(spark, "/mnt/tokyo-olympic/raw-data/entriesgender.csv")
medals = load_csv_data(spark, "/mnt/tokyo-olympic/raw-data/medals.csv")
teams = load_csv_data(spark, "/mnt/tokyo-olympic/raw-data/teams.csv")

In [0]:
medals.printSchema()
athletes.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
# Verify by displaying the first few rows of each DataFrame

athletes.show(5)
coaches.show(5)
entriesgender.show(5)
medals.show(5)
teams.show(5)

+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|C�te d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows

+-------------------+------+----+-----+
|         Discipline|Female|Male|Tota

In [0]:
# Have a full picture of the athletes DataFrame

athletes.printSchema()
athletes.describe().show()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

+-------+-----------------+-----------+--------------+
|summary|       PersonName|    Country|    Discipline|
+-------+-----------------+-----------+--------------+
|  count|            11085|      11085|         11085|
|   mean|             null|       null|          null|
| stddev|             null|       null|          null|
|    min|  AALERUD Katrine|Afghanistan|3x3 Basketball|
|    max|von MARTELS Chris|   Zimbabwe|     Wrestling|
+-------+-----------------+-----------+--------------+



In [0]:
# Fill in missing values

coaches = coaches.fillna({'Event': 'Unknown'})

In [0]:
coaches.filter(coaches['Country'].rlike('[^\x00-\x7F]')).show()

+----------------+-------------+----------+-------+
|            Name|      Country|Discipline|  Event|
+----------------+-------------+----------+-------+
|    ADAMA Cherif|C�te d'Ivoire|  Football|Unknown|
|HAIDARA Soualiho|C�te d'Ivoire|  Football|    Men|
|   JEROME Essapa|C�te d'Ivoire|  Football|Unknown|
|   KOFFI Anthony|C�te d'Ivoire|  Football|Unknown|
|NEVES Christophe|C�te d'Ivoire|  Football|Unknown|
+----------------+-------------+----------+-------+



In [0]:
# Write the transformed data to a transformed directory

athletes.repartition(1).write.mode('overwrite').option("header", "true").csv("/mnt/tokyo-olympic/transformed-data/athletes")
coaches.repartition(1).write.mode('overwrite').option("header", "true").csv("/mnt/tokyo-olympic/transformed-data/coaches")
entriesgender.repartition(1).write.mode('overwrite').option("header", "true").csv("/mnt/tokyo-olympic/transformed-data/entriesgender")
medals.repartition(1).write.mode('overwrite').option("header", "true").csv("/mnt/tokyo-olympic/transformed-data/medals")
teams.repartition(1).write.mode('overwrite').option("header", "true").csv("/mnt/tokyo-olympic/transformed-data/teams")
